In [1]:
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim

torch.set_printoptions(edgeitems=2, linewidth=75)

## 6.2.1 forward 대신 _ _ call _ _ 사용하기

In [ ]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # <1>
t_u = torch.tensor(t_u).unsqueeze(1) # <1>

t_u.shape

In [5]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([ 8,  0,  9,  3,  4,  7,  1,  5, 10]), tensor([2, 6]))

In [6]:
t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

## 6.2.2 다시 선형 모델로
: nn.Linear는 3개의 인자를 받음 
- 입력 피처 수
- 출력 피처 수
- 모델이 편향값을 포함하는지 default = True

In [8]:
import torch.nn as nn

linear_model = nn.Linear(1, 1) # 인스턴스화하여 함수처럼 사용 가능 
linear_model(t_un_val)

tensor([[-1.1837],
        [-0.8704]], grad_fn=<AddmmBackward0>)

In [9]:
linear_model.weight

Parameter containing:
tensor([[-0.1289]], requires_grad=True)

In [10]:
linear_model.bias

Parameter containing:
tensor([-0.4333], requires_grad=True)

In [26]:
x = torch.ones(1)
linear_model(x)

tensor([-0.8786], grad_fn=<ViewBackward0>)

In [27]:
# 배치 입력 만들기
x = torch.ones(10, 1)
linear_model(x)

tensor([[-0.8786],
        [-0.8786],
        [-0.8786],
        [-0.8786],
        [-0.8786],
        [-0.8786],
        [-0.8786],
        [-0.8786],
        [-0.8786],
        [-0.8786]], grad_fn=<AddmmBackward0>)

# 배치 최적화
- 연산량을 충분히 크게 만들어 준비한 자원을 최댛나 활용하기 위함

In [28]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [29]:
t_c, t_c.shape

(tensor([ 0.5000, 14.0000, 15.0000, 28.0000, 11.0000,  8.0000,  3.0000,
         -4.0000,  6.0000, 13.0000, 21.0000]),
 torch.Size([11]))

In [30]:
t_u, t_u.shape

(tensor([35.7000, 55.9000, 58.2000, 81.9000, 56.3000, 48.9000, 33.9000,
         21.8000, 48.4000, 60.4000, 68.4000]),
 torch.Size([11]))

In [31]:
t_c = t_c.unsqueeze(1) # 1번 축 차원 추가
t_u = t_u.unsqueeze(1) # 1번 축 차원 추가

t_u.shape

torch.Size([11, 1])

In [32]:
linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(
    linear_model.parameters(), lr=0.01
)

In [33]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[0.3348]], requires_grad=True),
 Parameter containing:
 tensor([0.4058], requires_grad=True)]

In [34]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [35]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

linear_model = nn.Linear(1, 1) # <1>
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(
    n_epochs = 3000, 
    optimizer = optimizer,
    model = linear_model,
    loss_fn = loss_fn,
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_c_train,
    t_c_val = t_c_val)

print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training loss 348.5883, Validation loss 213.8441
Epoch 1000, Training loss 3.8586, Validation loss 1.7329
Epoch 2000, Training loss 2.8136, Validation loss 3.9626
Epoch 3000, Training loss 2.7800, Validation loss 4.5136

Parameter containing:
tensor([[5.4907]], requires_grad=True)
Parameter containing:
tensor([-18.3005], requires_grad=True)


In [36]:
linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(
    n_epochs = 3000, 
    optimizer = optimizer,
    model = linear_model,
    loss_fn = nn.MSELoss(), 
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_c_train,
    t_c_val = t_c_val)

print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training loss 243.8095, Validation loss 147.7178
Epoch 1000, Training loss 3.7076, Validation loss 1.8727
Epoch 2000, Training loss 2.8088, Validation loss 4.0095
Epoch 3000, Training loss 2.7798, Validation loss 4.5227

Parameter containing:
tensor([[5.4921]], requires_grad=True)
Parameter containing:
tensor([-18.3086], requires_grad=True)
